In [1]:
import time
import numpy as np
import dask.array as da

In [2]:
def maxpo(s):
    #sx,sy,sxy = v[0],v[1],v[2]
    return (s[0]+s[1]) / 2 + (((s[0]-s[1])/2)**2 + s[2] ** 2) ** 0.5
def minpo(s):
    #sx,sy,sxy = v[0],v[1],v[2]
    return (s[0]+s[1]) / 2 - (((s[0]-s[1])/2)**2 + s[2] ** 2) ** 0.5

In [3]:
def maxp(s):
    #sx,sy,sxy = v[0],v[1],v[2]
    return (s[:,0]+s[:,1]) / 2 + (((s[:,0]-s[:,1])/2)**2 + s[:,2] ** 2) ** 0.5
def minp(s):
    #sx,sy,sxy = v[0],v[1],v[2]
    return (s[:,0]+s[:,1]) / 2 - (((s[:,0]-s[:,1])/2)**2 + s[:,2] ** 2) ** 0.5

## Variables

In [5]:
unilcs = 300
elms = 100000
clcs = 20
np.random.seed(0)

## Numpy

In [6]:
ar123 = np.random.random((unilcs,elms,3))#.astype(np.float32)
arlc = np.random.random((unilcs,clcs))
print(ar123.shape)
print(ar123.dtype)

(300, 100000, 3)
float64


In [7]:
ar123t = ar123.T

In [8]:
start = time.time()
res_t = ar123t.dot(arlc)
print("Time multi = %s" % (time.time() - start))

Time multi = 1.6310899257659912


In [9]:
ar = res_t.T

In [10]:
#amaxp = np.apply_along_axis(maxpo,-1,ar)
#aminp = np.apply_along_axis(minpo,-1,ar)

In [11]:
start = time.time()
amaxabs = np.zeros(shape=(clcs,elms))
for i in range(clcs):
    #amaxp = np.apply_along_axis(maxpo,-1,ar[i])
    #aminp = np.apply_along_axis(minpo,-1,ar[i])
    amaxp, aminp = maxp(ar[i]), minp(ar[i])
    x1, x2 = abs(amaxp), abs(aminp)
    amaxabs[i] = np.maximum.reduce([x1,x2])
print("Time maxP = %s" % (time.time() - start))

Time maxP = 0.19830894470214844


In [12]:
index = np.argmax(amaxabs, axis=0)

In [13]:
x = np.indices(index.shape)

In [14]:
ar2 = ar[index,x]

## dask

In [6]:
chunksize = 10000

In [7]:
ar123 = da.random.random((unilcs,elms,3),chunks=(unilcs, chunksize,3))
arlc = da.random.random((unilcs,clcs))

In [8]:
ar123t = ar123.T

In [9]:
start = time.time()
res_t = ar123t.dot(arlc).compute()
print("Time multi = %s" % (time.time() - start))

Time multi = 102.86254978179932


In [10]:
ar = res_t.T

In [11]:
start = time.time()
data_abs = []
for i in range(clcs):
    #chlen = ar[i].chunksize[0]
    #amaxp = np.apply_along_axis(maxpo,-1,ar[i], dtype=ar[i].dtype, shape=(chlen,)).compute()
    #aminp = np.apply_along_axis(minpo,-1,ar[i], dtype=ar[i].dtype, shape=(chlen,)).compute()
    amaxp, aminp = maxp(ar[i]), minp(ar[i])
    #print(time.time() - start, "step1")
    x1, x2 = abs(amaxp), abs(aminp)
    #print(time.time() - start, "step2")
    # x1, x2 = npo.asarray(abs(amaxp)), npo.asarray(abs(aminp))
    x1x2max = np.maximum(x1,x2)
    #print(time.time() - start, "step3")
    data_abs.append(x1x2max)
    #print(time.time() - start, "step4")
amaxabs = np.stack(data_abs, axis=0)
print("Time maxP = %s" % (time.time() - start))

Time maxP = 19.97642683982849


In [12]:
index = np.argmax(amaxabs, axis=0)
#indexd = da.from_array(index)
#indexd

In [13]:
x = np.indices(index.shape)

In [17]:
ar[index,x].shape

(1, 10000000, 3)